In [1]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(
        rotation_range=15,
        width_shift_range=0.0,
        height_shift_range=0.0,
        shear_range=0.1,
        zoom_range=0.2,
        vertical_flip=True,
        fill_mode='constant',zca_whitening=True, zca_epsilon=1e-5,featurewise_center=True)

img = load_img('train/glaucoma_resized/g0018.jpg')  # this is a PIL image
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
i = 0
for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='train/final_glu_aug', save_prefix='new', save_format='jpg'):
    i += 1
    if i > 6:
        break  # otherwise the generator would loop indefinitely

Using TensorFlow backend.
/home/subodh/anaconda3/lib/python3.6/site-packages/keras_preprocessing/image.py:959: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/home/subodh/anaconda3/lib/python3.6/site-packages/keras_preprocessing/image.py:978: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


In [3]:
import glob
import numpy as np
from PIL import *
import PIL.Image

In [4]:
filelist = glob.glob('train/final_glu_aug/*.jpg')
glu = np.array([np.array(Image.open(fname)) for fname in filelist])
print(glu.shape)

(357, 605, 700, 3)


In [5]:
filelist = glob.glob('train/final_non_glu_aug/*.jpg')
nonglu = np.array([np.array(Image.open(fname)) for fname in filelist])
print(nonglu.shape)

(357, 605, 700, 3)


In [6]:
X = np.concatenate((glu,nonglu),axis=0)
print(X.shape)

(714, 605, 700, 3)


In [7]:
gluL = np.array([1 for i in range(357)])
nongluL = np.array([0 for i in range(357)])
y = np.concatenate((gluL,nongluL),axis=0)
print(y.shape)

(714,)


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(478, 605, 700, 3) (236, 605, 700, 3) (478,) (236,)


In [46]:
from keras.models import Sequential
from keras.layers import Activation, Conv2D, MaxPooling2D, Flatten, Dropout,Dense
model = Sequential()

model.add(Conv2D(16, (3,3), padding='same', input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3,3),padding='valid'))
model.add(Activation('relu'))

model.add(Conv2D(32, (3,3), strides =2))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(256, (3,3), strides =2))
model.add(Activation('relu'))


model.add(Flatten())
model.add(Dense(32))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))


In [47]:
model.output_shape

(None, 1)

In [48]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_70 (Conv2D)           (None, 605, 700, 16)      448       
_________________________________________________________________
activation_89 (Activation)   (None, 605, 700, 16)      0         
_________________________________________________________________
conv2d_71 (Conv2D)           (None, 603, 698, 32)      4640      
_________________________________________________________________
activation_90 (Activation)   (None, 603, 698, 32)      0         
_________________________________________________________________
conv2d_72 (Conv2D)           (None, 301, 348, 32)      9248      
_________________________________________________________________
activation_91 (Activation)   (None, 301, 348, 32)      0         
_________________________________________________________________
max_pooling2d_37 (MaxPooling (None, 150, 174, 32)      0         
__________

In [49]:
# from keras import optimizers
# sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
# model.compile(loss='mean_squared_error', optimizer=sgd, metrics = ['accuracy'])
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [20]:
# model.fit(X_train, y_train, batch_size=8, epochs=30, verbose=2, validation_data=(X_test, y_test))
model.fit(X_train, y_train, epochs=10, batch_size=32,validation_data=(X_test, y_test))

Train on 187 samples, validate on 93 samples
Epoch 1/10
187/187 [==============================] - 994s 5s/step - loss: 1.4735 - acc: 0.8984 - val_loss: 1.9064 - val_acc: 0.8817
Epoch 2/10
187/187 [==============================] - 999s 5s/step - loss: 1.5505 - acc: 0.9037 - val_loss: 1.9064 - val_acc: 0.8817
Epoch 3/10
187/187 [==============================] - 992s 5s/step - loss: 1.5517 - acc: 0.9037 - val_loss: 1.9064 - val_acc: 0.8817
Epoch 4/10
187/187 [==============================] - 985s 5s/step - loss: 1.4653 - acc: 0.9091 - val_loss: 1.9064 - val_acc: 0.8817
Epoch 5/10
187/187 [==============================] - 987s 5s/step - loss: 1.4791 - acc: 0.9037 - val_loss: 1.9064 - val_acc: 0.8817
Epoch 6/10
187/187 [==============================] - 983s 5s/step - loss: 1.4653 - acc: 0.9091 - val_loss: 1.9064 - val_acc: 0.8817
Epoch 7/10
187/187 [==============================] - 984s 5s/step - loss: 1.4653 - acc: 0.9091 - val_loss: 1.9064 - val_acc: 0.8817
Epoch 8/10
187/187 [====

In [21]:
score = model.evaluate(X_test, y_test, batch_size=4)
print(score)

93/93 [==============================] - 134s 1s/step
[1.9064414567466919, 0.8817204301075269]


In [22]:
model.predict(X_test, batch_size=8)
model.predict_classes(X_test,batch_size=8)

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
    